# Fixing the error returned by testing the multihcannel localisation in btrack

In [1]:
import os
import glob
from octopusheavy import DaskOctopusHeavyLoader
import napari
from skimage.io import imshow,  imsave, imread
import napari
import btrack
from tqdm.auto import tqdm
import dask.array as da

# Loading images

Define root path and individual experiment IDs

In [2]:
root_path = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy'

In [3]:
expt_IDs = ['mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_262_/images/',
        ]

## Check to see what channels each expt has

In [4]:
for i, expt in enumerate(expt_IDs):
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    if 'MASK1' in [channel.name for channel in images.channels]:
        images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
        print(i, images.channels)

0 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]


In [5]:
images['MASK1']

dask.array<stack, shape=(81, 2304, 2304), dtype=uint8, chunksize=(1, 2304, 2304), chunktype=numpy.ndarray>

## Creating a multichannel dask array to feed to localisation

In [6]:
multichannel_stack = da.stack([images['CH1'], images['CH2'], images['CH3'], images['CH4']], axis = -1)

In [7]:
images['CH1'].shape

(81, 2304, 2304)

In [8]:
multichannel_stack.shape

(81, 2304, 2304, 4)

In [9]:
multichannel_stack.ndim

4

### Creating a pseudo 3D (zyx) stack to test

In [10]:
stack = da.stack([images['CH1'] for i in range(0,25)], axis = 1)

In [11]:
stack.shape

(81, 25, 2304, 2304)

### Defining properties to measure similarities of z-slices

In [12]:
props = ('axis_major_length', 
         'axis_minor_length', 
         'eccentricity', 
         'area', 
         #'intensity_image', 
         'orientation',
         'mean_intensity')

In [17]:
objects = btrack.utils.segmentation_to_objects(
            images['MASK1'], 
            multichannel_stack,
            multichannel_input=True,
            properties = props,#('area', 'mean_intensity', 'intensity_image'), 
#             use_weighted_centroid = False
        )

[INFO][2022/12/02 04:39:51 PM] Localizing objects from segmentation...


TypeError: Weighted centroids cannot be used for multichannel images yeah.

In [18]:
objects = btrack.utils.segmentation_to_objects(
            images['MASK1'], 
            multichannel_stack,
            properties = props,#('area', 'mean_intensity', 'intensity_image'), 
#             use_weighted_centroid = False
        )

[INFO][2022/12/02 04:40:59 PM] Localizing objects from segmentation...
[INFO][2022/12/02 04:40:59 PM] Found intensity_image data
[INFO][2022/12/02 04:40:59 PM] Calculating weighted centroids using intensity_image


ValueError: setting an array element with a sequence.

In [16]:
objects = btrack.utils.segmentation_to_objects(
            images['MASK1'], 
            multichannel_stack,
            multichannel_input=False,
            properties = props,#('area', 'mean_intensity', 'intensity_image'), 
#             use_weighted_centroid = False
        )

[INFO][2022/12/02 04:39:02 PM] Localizing objects from segmentation...
[INFO][2022/12/02 04:39:02 PM] Found intensity_image data
[INFO][2022/12/02 04:39:02 PM] Calculating weighted centroids using intensity_image


ValueError: setting an array element with a sequence.

In [15]:
objects = btrack.utils.segmentation_to_objects(
            images['MASK1'], 
            multichannel_stack,
#             multichannel_input=False,
            properties = props,#('area', 'mean_intensity', 'intensity_image'), 
#             use_weighted_centroid = False
        )

[INFO][2022/12/02 04:38:25 PM] Localizing objects from segmentation...
[INFO][2022/12/02 04:38:25 PM] Found intensity_image data
[INFO][2022/12/02 04:38:25 PM] Calculating weighted centroids using intensity_image
/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/skimage/measure/_regionprops.py:594: RuntimeWarning: invalid value encountered in true_divide
  tuple(M[_get_element(axis)] / M0 for axis in range(self._ndim)))


ValueError: setting an array element with a sequence.

In [14]:
objects = btrack.utils.segmentation_to_objects(
            images['MASK1'], 
            multichannel_stack,
            multichannel_input=True,
            properties = props,#('area', 'mean_intensity', 'intensity_image'), 
            use_weighted_centroid = False
        )

[INFO][2022/12/02 04:36:42 PM] Localizing objects from segmentation...
[INFO][2022/12/02 04:36:42 PM] Found intensity_image data

KeyboardInterrupt



# The issue:

I need a way of discerning whether the input intensity image is multichannel or not... 

I cannot just read the number of dimensions as it could be a 3D volume.

# I think if this one doesn't work it's due to the filename... but surely because I'm reading images from that address then the filepath works...?

In [15]:
for i, expt in tqdm(enumerate(expt_IDs), total = len(expt_IDs)):
    print(i)
    if i == 0:
        continue
    props = ('axis_major_length', 
         'axis_minor_length', 
         'eccentricity', 
         'area', 
         #'intensity_image', 
         'orientation',
         'mean_intensity')
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    multichannel_stack = da.stack([images['CH1'], images['CH2'], images['CH3'], images['CH4']], axis = -1)
    if 'MASK1' in [channel.name for channel in images.channels]: 
        ### create filename for tracks and objects 
        dirname = os.path.dirname(images.files('CH1')[0]).replace('images','')
        objects_fn = os.path.join(dirname, 'objects.hdf5')
        tracks_fn = os.path.join(dirname, 'tracks.hdf5')
        ### try saving locally instead of online
        tracks_fn = tracks_fn.split('Light microscopy/')[-1].replace('/', '_')
        objects_fn = objects_fn.split('Light microscopy/')[-1].replace('/', '_')
        objects = btrack.utils.segmentation_to_objects(
            images['MASK1'], 
            multichannel_stack,
            properties = props,#('area', 'mean_intensity', 'intensity_image'), 
            use_weighted_centroid = False
        )
        ### prune objects 
        objects = [o for o in objects if o.properties['area']>50]
        ### save out objects
        with btrack.dataio.HDF5FileHandler(
             objects_fn, 'w', obj_type='obj_type_1',
        ) as hdf:
            hdf.write_segmentation(images['MASK1'])
            hdf.write_objects(objects)
        
        ### redefine properties as multichannel image was measured
        props = ('axis_major_length', 
             'axis_minor_length', 
             'eccentricity', 
             'area', 
             #'intensity_image', 
             'orientation',
             'mean_intensity-0',
             'mean_intensity-1',
             'mean_intensity-2',
             'mean_intensity-3',)
        # initialise a tracker session using a context manager
        with btrack.BayesianTracker() as tracker:
            # configure the tracker using a config file
            tracker.configure('/home/dayn/analysis/BayesianTracker/models/particle_config.json')
            tracker.verbose = True
            ### set max search radius
            tracker.max_search_radius = 100
            # use visual features to track
            tracker.features = props
            # append the objects to be tracked
            tracker.append(objects)
            # set the volume (Z axis volume limits default to [-1e5, 1e5] for 2D data)
            tracker.volume=((0, 2304), (0, 2304), (-1e5, 1e5))
            # track them (in interactive mode)
#             tracker.track_interactive(step_size=100)
            tracker.track(tracking_updates =['visual', 'motion'], step_size=10)
            # generate hypotheses and run the global optimizer
            tracker.optimize()
            # get the tracks as a python list
            tracks = tracker.tracks
#             # filter tracks
#             tracks = [track for track in tracks if len(track) >= 3]
            # optional: get the data in a format for napari
            #   data, properties, graph = tracker.to_napari()
            tracker.export(tracks_fn, obj_type = 'obj_type_1')
                        

  0%|          | 0/3 [00:00<?, ?it/s]

0
1


[INFO][2022/12/02 11:06:17 AM] Localizing objects from segmentation...
[INFO][2022/12/02 11:06:17 AM] Found intensity_image data
[INFO][2022/12/02 11:08:43 AM] Objects are of type: <class 'dict'>
[INFO][2022/12/02 11:08:45 AM] ...Found 89178 objects in 81 frames.
[INFO][2022/12/02 11:08:45 AM] Opening HDF file: mono-culture_iAT2AT1_for analysis_20221125_DAPI-CAV1-proSPC-ZO1_Day7_static__20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_130__objects.hdf5...
[INFO][2022/12/02 11:09:13 AM] Writing objects/obj_type_1
[INFO][2022/12/02 11:09:13 AM] Writing labels/obj_type_1
[INFO][2022/12/02 11:09:13 AM] Loading objects/obj_type_1 (82669, 5) (82669 filtered: None)
[INFO][2022/12/02 11:09:14 AM] Writing properties/obj_type_1/axis_major_length (82669,)
[INFO][2022/12/02 11:09:14 AM] Writing properties/obj_type_1/axis_minor_length (82669,)
[INFO][2022/12/02 11:09:14 AM] Writing properties/obj_type_1/eccentricity (82669,)
[INFO][2022/12/02 11:09:14 AM] Writing properties/obj_type_1/area (8266

GLPK Integer Optimizer 5.0
42284 rows, 35645 columns, 50148 non-zeros
35645 integer variables, all of which are binary
Preprocessing...
21142 rows, 35645 columns, 50148 non-zeros
35645 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 21142
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
21142 rows, 35645 columns, 50148 non-zeros
*     0: obj =   1.279688888e+05 inf =   0.000e+00 (7517)
Perturbing LP to avoid stalling [1074]...
*  6264: obj =   9.043003531e+04 inf =   0.000e+00 (335) 2
Removing LP perturbation [6603]...
*  6603: obj =   9.025442271e+04 inf =   0.000e+00 (0) 1
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  6603: mip =     not found yet >=              -inf        (1; 0)
+  6603: >>>>>   9.025442271e+04 >=   9.025442271e+04   0.0% (1; 0)
+  6603: mip =   9

[WARNING][2022/12/02 11:37:59 AM] Changing HDF filename to mono-culture_iAT2AT1_for analysis_20221125_DAPI-CAV1-proSPC-ZO1_Day7_static__20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_130__tracks.h5
[INFO][2022/12/02 11:37:59 AM] Opening HDF file: mono-culture_iAT2AT1_for analysis_20221125_DAPI-CAV1-proSPC-ZO1_Day7_static__20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_130__tracks.h5...
[INFO][2022/12/02 11:38:00 AM] Writing objects/obj_type_1
[INFO][2022/12/02 11:38:00 AM] Writing labels/obj_type_1
[INFO][2022/12/02 11:38:00 AM] Loading objects/obj_type_1 (82669, 5) (82669 filtered: None)
[INFO][2022/12/02 11:38:01 AM] Writing properties/obj_type_1/axis_major_length (82669,)
[INFO][2022/12/02 11:38:01 AM] Writing properties/obj_type_1/axis_minor_length (82669,)
[INFO][2022/12/02 11:38:01 AM] Writing properties/obj_type_1/eccentricity (82669,)
[INFO][2022/12/02 11:38:01 AM] Writing properties/obj_type_1/area (82669,)
[INFO][2022/12/02 11:38:01 AM] Writing properties/obj_type_1

2


[INFO][2022/12/02 11:38:03 AM] Localizing objects from segmentation...
[INFO][2022/12/02 11:38:03 AM] Found intensity_image data
[INFO][2022/12/02 11:40:36 AM] Objects are of type: <class 'dict'>
[INFO][2022/12/02 11:40:37 AM] ...Found 64389 objects in 81 frames.
[INFO][2022/12/02 11:40:37 AM] Opening HDF file: mono-culture_iAT2AT1_for analysis_20221125_DAPI-CAV1-proSPC-ZO1_Day14_static__20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_253__objects.hdf5...
[INFO][2022/12/02 11:41:09 AM] Writing objects/obj_type_1
[INFO][2022/12/02 11:41:09 AM] Writing labels/obj_type_1
[INFO][2022/12/02 11:41:09 AM] Loading objects/obj_type_1 (58357, 5) (58357 filtered: None)
[INFO][2022/12/02 11:41:10 AM] Writing properties/obj_type_1/axis_major_length (58357,)
[INFO][2022/12/02 11:41:10 AM] Writing properties/obj_type_1/axis_minor_length (58357,)
[INFO][2022/12/02 11:41:10 AM] Writing properties/obj_type_1/eccentricity (58357,)
[INFO][2022/12/02 11:41:10 AM] Writing properties/obj_type_1/area (583

GLPK Integer Optimizer 5.0
35676 rows, 29549 columns, 41260 non-zeros
29549 integer variables, all of which are binary
Preprocessing...
17838 rows, 29549 columns, 41260 non-zeros
29549 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 17838
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
17838 rows, 29549 columns, 41260 non-zeros
*     0: obj =   1.013169624e+05 inf =   0.000e+00 (5574)
Perturbing LP to avoid stalling [1274]...
Removing LP perturbation [4981]...
*  4981: obj =   7.250326770e+04 inf =   0.000e+00 (0) 1
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  4981: mip =     not found yet >=              -inf        (1; 0)
+  4981: >>>>>   7.250326770e+04 >=   7.250326770e+04   0.0% (1; 0)
+  4981: mip =   7.250326770e+04 >=     tree is empty   0.0% (0; 1)
INTEGER O

[WARNING][2022/12/02 12:00:32 PM] Changing HDF filename to mono-culture_iAT2AT1_for analysis_20221125_DAPI-CAV1-proSPC-ZO1_Day14_static__20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_253__tracks.h5
[INFO][2022/12/02 12:00:32 PM] Opening HDF file: mono-culture_iAT2AT1_for analysis_20221125_DAPI-CAV1-proSPC-ZO1_Day14_static__20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_253__tracks.h5...
[INFO][2022/12/02 12:00:33 PM] Writing objects/obj_type_1
[INFO][2022/12/02 12:00:33 PM] Writing labels/obj_type_1
[INFO][2022/12/02 12:00:33 PM] Loading objects/obj_type_1 (58357, 5) (58357 filtered: None)
[INFO][2022/12/02 12:00:33 PM] Writing properties/obj_type_1/axis_major_length (58357,)
[INFO][2022/12/02 12:00:33 PM] Writing properties/obj_type_1/axis_minor_length (58357,)
[INFO][2022/12/02 12:00:33 PM] Writing properties/obj_type_1/eccentricity (58357,)
[INFO][2022/12/02 12:00:33 PM] Writing properties/obj_type_1/area (58357,)
[INFO][2022/12/02 12:00:33 PM] Writing properties/obj_type

In [17]:
expt_IDs = ['co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/images/',
            'co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-VWF-iCAM1-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_70_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_150_/images/',]

In [18]:
for i, expt in tqdm(enumerate(expt_IDs), total = len(expt_IDs)):
    print(i)
    if i == 0:
        continue
    props = ('axis_major_length', 
         'axis_minor_length', 
         'eccentricity', 
         'area', 
         #'intensity_image', 
         'orientation',
         'mean_intensity')
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    multichannel_stack = da.stack([images['CH1'], images['CH2'], images['CH3'], images['CH4']], axis = -1)
    if 'MASK1' in [channel.name for channel in images.channels]: 
        ### create filename for tracks and objects 
        dirname = os.path.dirname(images.files('CH1')[0]).replace('images','')
        objects_fn = os.path.join(dirname, 'objects.hdf5')
        tracks_fn = os.path.join(dirname, 'tracks.hdf5')
        ### try saving locally instead of online
        tracks_fn = tracks_fn.split('Light microscopy/')[-1].replace('/', '_')
        objects_fn = objects_fn.split('Light microscopy/')[-1].replace('/', '_')
        objects = btrack.utils.segmentation_to_objects(
            images['MASK1'], 
            multichannel_stack,
            properties = props,#('area', 'mean_intensity', 'intensity_image'), 
            use_weighted_centroid = False
        )
        ### prune objects 
        objects = [o for o in objects if o.properties['area']>50]
        ### save out objects
        with btrack.dataio.HDF5FileHandler(
             objects_fn, 'w', obj_type='obj_type_1',
        ) as hdf:
            hdf.write_segmentation(images['MASK1'])
            hdf.write_objects(objects)
        
        ### redefine properties as multichannel image was measured
        props = ('axis_major_length', 
             'axis_minor_length', 
             'eccentricity', 
             'area', 
             #'intensity_image', 
             'orientation',
             'mean_intensity-0',
             'mean_intensity-1',
             'mean_intensity-2',
             'mean_intensity-3',)
        # initialise a tracker session using a context manager
        with btrack.BayesianTracker() as tracker:
            # configure the tracker using a config file
            tracker.configure('/home/dayn/analysis/BayesianTracker/models/particle_config.json')
            tracker.verbose = True
            ### set max search radius
            tracker.max_search_radius = 100
            # use visual features to track
            tracker.features = props
            # append the objects to be tracked
            tracker.append(objects)
            # set the volume (Z axis volume limits default to [-1e5, 1e5] for 2D data)
            tracker.volume=((0, 2304), (0, 2304), (-1e5, 1e5))
            # track them (in interactive mode)
#             tracker.track_interactive(step_size=100)
            tracker.track(tracking_updates =['visual', 'motion'], step_size=10)
            # generate hypotheses and run the global optimizer
            tracker.optimize()
            # get the tracks as a python list
            tracks = tracker.tracks
#             # filter tracks
#             tracks = [track for track in tracks if len(track) >= 3]
            # optional: get the data in a format for napari
            #   data, properties, graph = tracker.to_napari()
            tracker.export(tracks_fn, obj_type = 'obj_type_1')
                        

  0%|          | 0/3 [00:00<?, ?it/s]

0
1


[INFO][2022/12/02 12:00:37 PM] Localizing objects from segmentation...
[INFO][2022/12/02 12:00:37 PM] Found intensity_image data
[INFO][2022/12/02 12:05:50 PM] Objects are of type: <class 'dict'>
[INFO][2022/12/02 12:05:51 PM] ...Found 63985 objects in 81 frames.
[INFO][2022/12/02 12:05:51 PM] Opening HDF file: co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-VWF-iCAM1-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_70__objects.hdf5...
[INFO][2022/12/02 12:06:32 PM] Writing objects/obj_type_1
[INFO][2022/12/02 12:06:32 PM] Writing labels/obj_type_1
[INFO][2022/12/02 12:06:32 PM] Loading objects/obj_type_1 (54035, 5) (54035 filtered: None)
[INFO][2022/12/02 12:06:33 PM] Writing properties/obj_type_1/axis_major_length (54035,)
[INFO][2022/12/02 12:06:33 PM] Writing properties/obj_type_1/axis_minor_length (54035,)
[INFO][2022/12/02 12:06:33 PM] Writing properties/obj_type_1/eccentricity (54035,)
[INFO][2022/12/02 12:06:33 PM] Writing propertie

GLPK Integer Optimizer 5.0
41128 rows, 36036 columns, 51508 non-zeros
36036 integer variables, all of which are binary
Preprocessing...
20564 rows, 36036 columns, 51508 non-zeros
36036 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 20564
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
20564 rows, 36036 columns, 51508 non-zeros
*     0: obj =   1.346829424e+05 inf =   0.000e+00 (10413)
Perturbing LP to avoid stalling [1052]...
*  5073: obj =   9.536215990e+04 inf =   0.000e+00 (3993)
Removing LP perturbation [9467]...
*  9467: obj =   8.156897322e+04 inf =   0.000e+00 (0) 5
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  9467: mip =     not found yet >=              -inf        (1; 0)
+  9467: >>>>>   8.156897322e+04 >=   8.156897322e+04   0.0% (1; 0)
+  9467: mip =   8

[WARNING][2022/12/02 12:14:02 PM] Changing HDF filename to co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-VWF-iCAM1-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_70__tracks.h5
[INFO][2022/12/02 12:14:02 PM] Opening HDF file: co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-VWF-iCAM1-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_70__tracks.h5...
[INFO][2022/12/02 12:14:03 PM] Writing tracks/obj_type_1
[WARNING][2022/12/02 12:14:03 PM] Removing tracks/obj_type_1.
[INFO][2022/12/02 12:14:03 PM] Writing dummies/obj_type_1
[INFO][2022/12/02 12:14:03 PM] Writing LBEP/obj_type_1
[INFO][2022/12/02 12:14:03 PM] Writing fates/obj_type_1
[INFO][2022/12/02 12:14:03 PM] Closing HDF file: co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-VWF-iCAM1-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_70__tracks.h5
[INFO][2022/12/02 12:14:03 PM] Ending BayesianTrack

2


[INFO][2022/12/02 12:14:04 PM] Localizing objects from segmentation...
[INFO][2022/12/02 12:14:04 PM] Found intensity_image data
[INFO][2022/12/02 12:16:42 PM] Objects are of type: <class 'dict'>
[INFO][2022/12/02 12:16:43 PM] ...Found 77706 objects in 81 frames.
[INFO][2022/12/02 12:16:43 PM] Opening HDF file: mono-culture_iAT2AT1_for analysis_20221125_DAPI-AQP5-proSPC-ZO1_Day7_static__20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_150__objects.hdf5...
[INFO][2022/12/02 12:17:13 PM] Writing objects/obj_type_1
[INFO][2022/12/02 12:17:13 PM] Writing labels/obj_type_1
[INFO][2022/12/02 12:17:13 PM] Loading objects/obj_type_1 (70817, 5) (70817 filtered: None)
[INFO][2022/12/02 12:17:13 PM] Writing properties/obj_type_1/axis_major_length (70817,)
[INFO][2022/12/02 12:17:13 PM] Writing properties/obj_type_1/axis_minor_length (70817,)
[INFO][2022/12/02 12:17:14 PM] Writing properties/obj_type_1/eccentricity (70817,)
[INFO][2022/12/02 12:17:14 PM] Writing properties/obj_type_1/area (7081

GLPK Integer Optimizer 5.0
41680 rows, 35405 columns, 49970 non-zeros
35405 integer variables, all of which are binary
Preprocessing...
20840 rows, 35405 columns, 49970 non-zeros
35405 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 20840
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
20840 rows, 35405 columns, 49970 non-zeros
*     0: obj =   1.240719098e+05 inf =   0.000e+00 (7988)
Perturbing LP to avoid stalling [988]...
*  6350: obj =   8.448660897e+04 inf =   0.000e+00 (640) 2
Removing LP perturbation [7020]...
*  7020: obj =   8.389569247e+04 inf =   0.000e+00 (0) 1
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  7020: mip =     not found yet >=              -inf        (1; 0)
+  7020: >>>>>   8.389569247e+04 >=   8.389569247e+04   0.0% (1; 0)
+  7020: mip =   8.

[WARNING][2022/12/02 12:38:49 PM] Changing HDF filename to mono-culture_iAT2AT1_for analysis_20221125_DAPI-AQP5-proSPC-ZO1_Day7_static__20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_150__tracks.h5
[INFO][2022/12/02 12:38:49 PM] Opening HDF file: mono-culture_iAT2AT1_for analysis_20221125_DAPI-AQP5-proSPC-ZO1_Day7_static__20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_150__tracks.h5...
[INFO][2022/12/02 12:38:50 PM] Writing tracks/obj_type_1
[WARNING][2022/12/02 12:38:50 PM] Removing tracks/obj_type_1.
[INFO][2022/12/02 12:38:50 PM] Writing dummies/obj_type_1
[INFO][2022/12/02 12:38:50 PM] Writing LBEP/obj_type_1
[INFO][2022/12/02 12:38:50 PM] Writing fates/obj_type_1
[INFO][2022/12/02 12:38:50 PM] Closing HDF file: mono-culture_iAT2AT1_for analysis_20221125_DAPI-AQP5-proSPC-ZO1_Day7_static__20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_150__tracks.h5
[INFO][2022/12/02 12:38:50 PM] Ending BayesianTracker session


# It won't let me save tracks to the server? But will let me save locally

In [33]:
tracks_fn

'/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/tracks.hdf5'

In [35]:
with btrack.dataio.HDF5FileHandler("tracks.h5", "w", obj_type="obj_type_1") as hdf:
    hdf.write_tracks(tracks)

[INFO][2022/12/02 09:22:48 AM] Opening HDF file: tracks.h5...
[INFO][2022/12/02 09:22:49 AM] Writing objects/obj_type_1
[INFO][2022/12/02 09:22:49 AM] Writing labels/obj_type_1
[INFO][2022/12/02 09:22:49 AM] Loading objects/obj_type_1 (59306, 5) (59306 filtered: None)
[INFO][2022/12/02 09:22:49 AM] Writing properties/obj_type_1/axis_major_length (59306,)
[INFO][2022/12/02 09:22:49 AM] Writing properties/obj_type_1/axis_minor_length (59306,)
[INFO][2022/12/02 09:22:49 AM] Writing properties/obj_type_1/eccentricity (59306,)
[INFO][2022/12/02 09:22:50 AM] Writing properties/obj_type_1/area (59306,)
[INFO][2022/12/02 09:22:50 AM] Writing properties/obj_type_1/orientation (59306,)
[INFO][2022/12/02 09:22:50 AM] Writing tracks/obj_type_1
[INFO][2022/12/02 09:22:50 AM] Writing LBEP/obj_type_1
[INFO][2022/12/02 09:22:50 AM] Writing fates/obj_type_1
[INFO][2022/12/02 09:22:50 AM] Closing HDF file: tracks.h5


#### testing unique fn 

In [49]:
tracks_fn.split('Light microscopy/')[-1].replace('/', '_')

'co-culture_iVECs+iAT2AT1_Folder_20220808_A2-A5_analysis_20221125_DAPI-SPC-PDPN-ZO1__20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60__tracks.hdf5'

In [37]:
tracks_fn

'/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/tracks.hdf5'

In [43]:
test_tracks_fn = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/users/dayn/test_tracks_output/tracks.hdf5'

In [44]:
with btrack.dataio.HDF5FileHandler(test_tracks_fn, "w", obj_type="obj_type_1") as hdf:
    hdf.write_tracks(tracks)

[INFO][2022/12/02 09:25:39 AM] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/users/dayn/test_tracks_output/tracks.hdf5...
[INFO][2022/12/02 09:25:39 AM] Writing objects/obj_type_1
[INFO][2022/12/02 09:25:39 AM] Writing labels/obj_type_1
[INFO][2022/12/02 09:25:39 AM] Loading objects/obj_type_1 (59306, 5) (59306 filtered: None)
[INFO][2022/12/02 09:25:41 AM] Writing properties/obj_type_1/axis_major_length (59306,)
[INFO][2022/12/02 09:25:41 AM] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/users/dayn/test_tracks_output/tracks.hdf5


OSError: [Errno 95] Unable to extend file properly, errno = 95, error message = 'operation not supported' (file write failed: time = Fri Dec  2 09:25:41 2022
, filename = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/users/dayn/test_tracks_output/tracks.hdf5', file descriptor = 66, errno = 95, error message = 'Operation not supported', buf = 0x5620987ad0c8, total write size = 3368, bytes this sub-write = 3368, bytes actually written = 18446744073709551615, offset = 0)

## Checking tracks prior to plotting single cell info

In [ ]:
import napari

In [ ]:
viewer = napari.Viewer()

viewer.add_image(ch0, colormap='blue', blending = 'additive')
viewer.add_image(ch3, colormap= 'gray', blending = 'additive')
viewer.add_image(ch1, colormap='red', blending = 'additive')
viewer.add_image(ch2, colormap= 'green', blending = 'additive')

viewer.add_labels(mask_stack)

viewer.add_tracks(ch1_data)
viewer.add_tracks(ch2_data)
